<a href="https://colab.research.google.com/github/elainedias16/TCC/blob/main/Copy_of_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large language models(LLM)

## Motivação

Em 2017, a publicação do artigo "Attention is All You Need" revolucionou o campo do Processamento de Linguagem Natural (NLP) com a introdução dos Transformers. Essa arquitetura neural, com sua capacidade de processar sequências de dados de forma mais eficiente, impulsionou o desenvolvimento de modelos de linguagem cada vez mais sofisticados e poderosos. A partir daí, os Grandes Modelos de Linguagem (LLMs) experimentaram um vasto crescimento, atraindo investimentos consideráveis e abrindo novas fronteiras para a área. Para se manter atualizado nesse cenário, é fundamental compreender o funcionamento dos LLM, de forma a aprimorar e desenvolver novas aplicações.

## Resultados Esperados



Neste laboratório, espera-se que os alunos compreendam os princípios básicos do funcionamento de um Grande Modelo de Linguagem. Para exemplificar, será apresentado um código em pequena escala de um LLM.

## Fundamentação teórica

## Código

O código abaixo realiza a instalação e importação das bibliotecas necessárias para o desenvolvimento do modelo LLM. A principal biblioteca utilizada será o PyTorch, que fornece os módulos essenciais para redes neurais e otimização, através dos pacotes torch.nn e torch.optim, respectivamente. Também são importadas as bibliotecas Dataset e DataLoader para a criação e manipulação de datasets e mini-lotes durante o treinamento. Além disso, a biblioteca Counter, da coleção padrão, e a nltk são utilizadas para contagem e pré processamento de dados, com nltk incluindo o método word_tokenize para tokenização.


Vale ressaltar que é possível criar um modelo LLM com outras bibliotecas, por exemplo, a Keras. A escolha da biblioteca Torch foi devido a maior familiaridade com a mesma.

In [82]:
!pip install transformers torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import torch.nn.functional as F

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Masked Self-Attention

Scale dot produt attetion:
https://paperswithcode.com/method/scaled

In [55]:
# import math
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

class Head(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.query = nn.Linear(config.d_model, config.head_dim, bias=config.bias)
    self.key = nn.Linear(config.d_model, config.head_dim, bias=config.bias)
    self.value = nn.Linear(config.d_model, config.head_dim, bias=config.bias)


  def forward(self, x):
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    return q, k, v



class MaskedSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads
        self.head_dim = config.head_dim
        self.dropout = nn.Dropout(config.dropout)
        self.d_model = config.d_model
        self.heads = nn.ModuleList([Head(config) for _ in range(config.num_heads)])
        self.output_linear = nn.Linear(config.d_model, config.d_model)
        assert self.head_dim * self.num_heads == self.d_model, "d_model must be divisible by num_heads"


    def forward(self, x, mask=None):
        # B, T, C = x.size()

        heads_output = []
        for head in self.heads:
            k, q, v = head(x)


            # Scaled dot-product attention
            scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

            if mask is not None:
                scores = scores.masked_fill(mask == 0, float('-inf'))

            attn_weights = F.softmax(scores, dim=-1)
            attn_weights = self.dropout(attn_weights)

            head_output = torch.matmul(attn_weights, v)
            heads_output.append(head_output)


        concatenated_output = torch.cat(heads_output, dim=-1)
        output = self.output_linear(concatenated_output)

        return output




### Feed Forward Neural Network


MLP

In [56]:
class FeedFoward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear1 = nn.Linear(config.d_model, 4 * config.d_model, bias=config.bias)
    self.activation = nn.ReLU()
    self.linear2 = nn.Linear(config.d_model * 4,  config.d_model, bias=config.bias)
    self.dropout = nn.Dropout(config.dropout)


  def forward(self, x):
    x = self.linear1(x)
    x = self.activation(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x



### Layer Norm

In [57]:
class LayerNorm(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.norm = nn.LayerNorm(config.d_model, config.bias)

  def forward(self, x):
    self.norm(x)
    return x

### One Decoder

In [58]:
class Decoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.ln_1 = LayerNorm(config)
    self.masked_self_attention = MaskedSelfAttention(config)
    self.ln_2 = LayerNorm(config)
    self.feed_forward = FeedFoward(config)

  # def forward(self, x, mask):
  #   x = self.ln_1(x)
  #   x = x + self.masked_self_attention(x, mask)
  #   x = self.ln_2(x)
  #   x = x + self.feed_forward(x)
  #   return x
  def forward(self, x):
    x = self.ln_1(x)
    x = x + self.masked_self_attention(x)
    x = self.ln_2(x)
    x = x + self.feed_forward(x)
    return x



### Transformer

https://medium.com/@hunter-j-phillips/positional-encoding-7a93db4109e6#:~:text=class%20PositionalEncoding(nn,self.dropout(x)
olhar posiitional

O código abaixo agrega os componentes criados anteriormente e acescenta os restantes para o funcionamento do modelo LLM.

Assim, na inicialização da classe, tem-se o parâmetro config, que são as configurações do modelo. Em seguida, são atribuídos os blocos e a camada de normalização que foram vistas anteriormente. Além disso, são criadas as camadas de embeddings e de posição dos tokens , além da camada de dropout e uma camada linear final.


Em seguida são criados dois métodos, o "forward" e o "generate". O método "forward" é responsável por definir o fluxo de dados pela rede, sendo seus parâmetros os input_ids e os targets. Os input_ids correspondem as palavras, mas quando chegam nesse ponto do código já estão em formato de tensores PyTorch. O parâmetro targets foi definido como optional (targets=None), para o caso do modelo no modo de teste, uma vez que só possível os targets no modo de treinamento.

Em seguida, é definido uma variável de device que recebe o mesmo device de onde estão os tensores de entrada (palavras => tokens => input_ids => tensor) . Isso é feito para garantir que os cálculos ocorram no mesmo dispostivo.

As variáveis B e T são utilizadas para armazenar o tamanho do batch e o comprimento da sequência, respectivamente. Após isso, o tensor de entrada é incorporado na camada de embeddings, de forma que cada ID no tensor passa a ter uma representação vetorial através da camada word_token_embedding. Além disso, a posição dos IDs é incorporada na camada de embeddings posicional (position_embedding), utilizando o comprimento da sequência como parâmetro para gerar a incorporação posicional. Essa incorporação posicional permite que o modelo capture a ordem dos tokens na sequência. As representações vetoriais dos tokens e suas posições são somadas (tok_emb + pos_emb) para gerar a entrada final x. Essa soma combina o significado semântico dos tokens com sua posição na sequência, o que permite ao modelo aprender tanto o contexto quanto a ordem relativa dos tokens.  Após isso, a soma (tok_emb + pos_emb) passa por uma camada de dropout para reduzir a chance de overting.


Após a camanda de dropout, os dados passam pelos blocos que foram explicados anteriormente. Assim, os dados por seis blocos, os quais foram definidos nas configurações. Em seguida, os dados passam pela camada de normalização e pela camada linear final, a qual tem como saída os logits, que são os valores brutos da classificação gerada.

No caso do modelo estar em treinamento, é adicionado uma linha para realizar o cálculo de entropia cruzada, retornando tanto os logits quanto a loss (perda).


O método de "generate" é utilizado para o modo de teste do modelo. Esse método realiza um loop com o número máximo de tokens a serem gerados, chamando o método de "forward" em cada iteração. É criado um array de output_ids para armazenar os outputs do método de forward. Os outsputs do método forward são os logits, por causa disso, é preciso passar esses logits por uma função de softmax de forma a gerar um vetor de probabilidades do próximo token da sentença, sendo que o token de maior probabilidade é escolhido e armazenado no array output_ids. Em seguida, o token gerado é concatenado à sentença de entrada formando um novo input_ids e assim o loop continua até o máximo de número de tokens ser atingido. Por fim, o array de output_ids é retornado e precisará passar pelo método de "decode" para ser interpreta em linguagem natural.

In [59]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import Adam
# from torch.utils.data import DataLoader, Dataset

class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.word_token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_embedding = nn.Embedding(config.max_length, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        self.blocks = nn.Sequential(*[Decoder(config) for _ in range(config.n_layer)])
        self.ln = LayerNorm(config)
        self.ll = nn.Linear(config.d_model, config.vocab_size, bias=False)

    def forward(self, input_ids, targets=None):
        device = input_ids.device
        B, T = input_ids.size()

        # Positional e token embed
        tok_emb = self.word_token_embedding(input_ids)
        pos_emb = self.position_embedding(torch.arange(T, device=device))
        x = self.dropout(tok_emb + pos_emb)

        # Transformer blocks
        x = self.blocks(x)

        # Norm layer
        x = self.ln(x)

        # Final layer
        logits = self.ll(x)

        if targets is not None:
            shift_logits = logits[:, :-1, :].contiguous()
            shift_targets = targets[:, 1:].contiguous()
            loss = F.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_targets.view(-1))
            return logits, loss

        return logits, None

    # def generate(self, input_ids, max_new_tokens):
    #     new_tokens = []

    #     for _ in range(0, max_new_tokens):
    #         input_ids_cond = input_ids[:, -self.config.block_size:]
    #         logits, _ = self.forward(input_ids_cond)
    #         logits = logits[:, -1, :]
    #         probs = F.softmax(logits, dim=-1)
    #         input_ids_next = torch.multinomial(probs, num_samples=1)
    #         new_tokens.append(input_ids_next)
    #         input_ids = torch.cat((input_ids, input_ids_next), dim=1)

    #     new_tokens = torch.cat(new_tokens, dim=1)
    #     print(f"aaaaa: {new_tokens}")
    #     return new_tokens


    def generate(self, input_ids, max_new_tokens):
      output_ids = []

      for _ in range(0, max_new_tokens):
          input_ids_cond = input_ids[:, -self.config.block_size:]
          logits, _ = self.forward(input_ids_cond)
          logits = logits[:, -1, :]
          probs = F.softmax(logits, dim=-1)
          input_ids_next = torch.multinomial(probs, num_samples=1)
          output_ids.append(input_ids_next)
          input_ids = torch.cat((input_ids, input_ids_next), dim=1)

      output_ids = torch.cat(output_ids, dim=1)
      print(f"aaaaa: {output_ids}")
      return output_ids


### Configurações do modelo

O código abaixo define os parâmetros gerais para o treinamento do modelo LLM. Assim, são definidos o dispositivo que será utilizado, o número máximo de tokens a serem gerados, o número de épocas de treinamento, a taxa de aprendizado do otimizador do modelo (será utilizado o otimizador Adam), o tamanho dos lotes de treinamento e o tamanho da sequência de entrada do modelo.



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_new_tokens = 50
epochs = 100
learning_rate = 0.001
batch_size = 8
SEQUENCE_LENGTH = 64

O código abaixo define os hiperparâmetros que definem a arquitetura do modelo. Assim, são definidos o número de cabeças para os cálculos de atenção, a dimensão do modelo, a dimensão de cada cabeça de atenção, a taxa de dropout para evitar overfitting, a presença ou não de bias, o tamanho do vocabulário, o tamanho máximo da sequência de entrada, o número de camadas do modelo e o tamanho do bloco de entrada.

In [61]:
class Config:
    num_heads = 2
    d_model = 64
    head_dim = 32
    dropout = 0.1
    bias = True
    vocab_size = 50257      # Len tokenizer (for now)
    # hidden_size = 1024
    max_length = 512
    n_layer = 6
    block_size = SEQUENCE_LENGTH
    # block_size = 1024
    # block_size = 32


config = Config()

### Treinamento

O código abaixo faz o download do dataset "alice.txt" .  Em seguida, é realiza a leitura do arquivo, utilizando a função open() para abrir o arquivo no modo de leitura ('r') e com codificação UTF-8. O conteúdo completo do arquivo é lido e armazenado na variável text.

In [62]:
# !wget https://raw.githubusercontent.com/elainedias16/TCC/main/alice_1.txt

# with open('alice_1.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

In [63]:
text = "Sofia era uma menina quieta e introspectiva, mas havia algo nela que todos sabiam: seu amor pelos livros. Desde muito pequena, tinha uma curiosidade insaciável pelo mundo ao seu redor. Aos cinco anos, já estava folheando livros com ilustrações, fascinada pelas imagens e pelas poucas palavras que conhecia. À medida que crescia, essa paixão só aumentava. Seus pais a encontravam, todos os dias, escondida em algum canto da casa com um livro nas mãos, como se estivesse em outro mundo. Na escola, Sofia não era a criança mais extrovertida, mas os livros lhe davam confiança. Enquanto os colegas brincavam no pátio, ela preferia a biblioteca. A bibliotecária, Dona Clara, rapidamente se tornou sua amiga e confidente. Dona Clara sabia exatamente quais livros indicar para cada fase de Sofia. Desde contos de fadas clássicos até aventuras fantásticas, cada novo livro era uma porta para um mundo cheio de magia e descobertas. Sofia gostava de se imaginar como as protagonistas das histórias que lia. Às vezes, ela era uma exploradora destemida, em outras, uma princesa corajosa que lutava contra o mal. Mas o que mais a fascinava eram as palavras. Ela não apenas lia, mas sentia cada frase, cada parágrafo como se fosse parte da sua própria vida. Isso a inspirou a começar a escrever suas próprias histórias. No começo, suas histórias eram simples, contos sobre princesas, dragões e reinos distantes. No entanto, à medida que crescia, seus escritos se tornaram mais profundos. Ela começou a explorar temas sobre amizade, coragem e superação. Aos 12 anos, já tinha um caderno cheio de histórias que criava, e sonhava, um dia, em publicar seus próprios livros. Para Sofia, ler não era apenas um hobby; era a chave que a conectava a um mundo infinito de imaginação e conhecimento."

O método abaizo faz a tokenização de um texto, além de tranformar os caracteres em caracteres minúsculos.

In [64]:
def tokenize(text):
  return word_tokenize(text.lower())

O código abaixo cria os dicionários de codificação e decodificação das palavras, além de retornar o tamanho do vocabulário. Para criar esses dicionários, é utilizado o método "Counter", que contabiliza a quantidade de  cada palavra no conjunto de tokens fornecidos. Em seguida, é criada uma lista auxiliar contendo as palavras únicas do vocabulário. O dicionário "word_to_int" é então gerado, associando cada palavra a um índice numérico com base em sua posição no vocabulário, e o dicionário "int_to_word" faz o mapeamento inverso, associando os índices às palavras correspondentes.

In [65]:
def build_vocab(tokens):
  word_counts = Counter(tokens)
  vocab = list(word_counts.keys())
  word_to_int = {word: i for i, word in enumerate(vocab)}
  int_to_word = {i: word for word, i in word_to_int.items()}

  return len(vocab), word_to_int, int_to_word

O código abaixo cria as amostras de entrada e saída para o modelo LLM. Uma vez que o dataset é apenas um conjunto de palavras, é necessário separar as palavras em data/target para que seja possível realizar o treinamento. Para isso, são passados os tokens , o dicionário de codificação de palavras e o tamanho da sequência de entrada do modelo.

Inicialmente, os tokens são codificadores em IDs e são criadas amostras (samples) desses tokens. As amostras são criadas a partir da lista "encoded" criada anteriormente, no formato "i : i +  sequence_length + 1", que significa que para cada posição de i dos tokens será criada uma amostra de tamanho de sequence_length + 1, começando a sequência de tokens a partir do token i. Essas amostras são criadas em um loop de tamanho "len(encoded) - sequence_length", para que não "estore" o tamanho da lista ao acessar tokens muito perto do final, pois não é possível criar uma amostra de tamanho "sequence_length + 1", acessando o penúltimo token da lista, por exemplo, por isso não é possível iterar sobre toda a lista de tokens para criar amostras.



Após isso, todos os tokens de cada amostra (sequence_length + 1) menos os últimos são armazenados na variável data e convertidos em um tensor, sendo que os últimos tokens de cada amostra são armazendos na variável targets e convertidos em um tensor.

Esse método codifca os texto para o treinamento.


In [66]:
def prepare_data(tokens, word_to_int, sequence_length=64):
  encoded = [word_to_int[token] for token in tokens]
  samples = [encoded[i:i + sequence_length + 1] for i in range(len(encoded) - sequence_length)]
  data = torch.tensor([sample[:-1] for sample in samples], dtype=torch.long)
  targets = torch.tensor([sample[1:] for sample in samples], dtype=torch.long)
  return data, targets

O código abaixo define a classe TextDataset, que herda da classe Dataset do PyTorch. O objetivo dessa classe é organizar o dataset em dois componentes: data e targets. Além disso, foi criado o método "getitem", que dado um índice, retorna uma tupla contendo o dado e o target correspondente. Também foi criado o método "len", que retorna o tamanho total do conjunto de dados. Posteriomente, essa classe será utilizada para instanciar um DataLoader.

In [67]:
class TextDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

O método abaixo realiza o treinamento do modelo. Ele recebe como parâmetros o modelo, o otimizador, o número de épocas de treinamento e o dataloader. Incialmente, o modelo é configurado em modo de treinamento e um loop é criado para iterar pelas épocas, com a perda total sendo inicializada em $0$. Para cada conjunto de input_ids e targets fornecido pelo dataloader, o modelo é chamado e calcula-se os logits e a perda. Em seguida, realiza-se a retropropagação da perda e a atualização dos pesos. A perda de cada lote é somada à perda total, e ao final de cada época, o valor médio da perda é exibido.

In [68]:
def train_model(model, dataloader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for input_ids, targets in dataloader:
            optimizer.zero_grad()
            logits, loss = model(input_ids, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

O código abaixo faz a instanciação do modelo e do otimizador que será utilizadado.

In [69]:
model = Transformer(config)
optimizer = Adam(model.parameters(), lr=learning_rate)

O código abaixo faz a tokenizaçao do dataset e chama o método de criar o vocabulário, o qual retorna os dicionários de codificação e decodificação das palavras e o tamanho do vocabulário. Em seguida, é ajustado o tamanho do vocabulário nas configurações do modelo , pois o tamanho deixado como default  é o tamanho do tokenizador gpt-tokenizer.

In [70]:
tokenize_text = tokenize(text)
vocab_size, word_to_int, int_to_word = build_vocab(tokenize_text)

config.vocab_size = vocab_size

O código abaixo chama o método "prepare_data" que separa o texto do dataset em data e targets, para que seja possível realizar o treinamento do modelo.

In [71]:
data, targets = prepare_data(tokenize_text, word_to_int)

O código abaixo define um objeto do tipo TextDataset, adequado para manipular dados textuais. Em seguida, é criado um DataLoader, que permite realizar o treinamento em mini-lotes (mini-batches). O tamanho de cada lote é determinado pela variável batch_size. O parâmetro shuffle=True indica que os dados serão embaralhados a cada época de treinamento, o que contribui para a melhor generalização do modelo.

In [72]:
dataset = TextDataset(data, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

O código abaixa chama o método de "train_model" que realiza o treinamento do modelo. Para rodar o modelo sem o treinamento, basta comentar a linha abaixo.

In [73]:
train_model(model, dataloader, optimizer, epochs=1)

Epoch 1, Loss: 6.170171439647675


### Testes

O código abaixo realiza a codificação das palavras de um texto. Primeiro, o texto é tokenizado, e em seguida percorre-se o dicionário de mapeamento de palavras (word_to_int) para converter cada token em seu respectivo ID. Caso o token não esteja no dicionário, é utilizado o ID correspondente ao token <pad>. Por fim, a lista de IDs é convertida em um tensor do tipo long do PyTorch, que no geral é formato esperado pelos modelos LLM, quando utiliza-se a biblioteca PyTorch.

Esse método é utilizado apenas para a codificação do prompt do usuário.

In [74]:
def encode(text, word_to_int):
    tokens = tokenize(text)
    if '<pad>' not in word_to_int:
        pad_id = 0
    else:
        pad_id = word_to_int['<pad>']
    encoded = [word_to_int.get(token, pad_id) for token in tokens]
    return torch.tensor(encoded, dtype=torch.long).unsqueeze(0)

O código abaixo decodifica os output_ids, que são a saída do modelo criado. Os parâmetros da função são tokens (representando os output_ids) e o dicionário de decodificação int_to_word. Como os tokens podem ter dimensões adicionais, é aplicado o método squeeze() o que facilita a iteração direta sobre os tokens.

Após a aplicação de squeeze(), cada token se torna um valor de dimensão única. Em seguida, o código verifica se o valor do token (obtido com token.item()) está presente no dicionário de decodificação int_to_word. Se o token estiver presente no dicionário, a palavra correspondente é adicionada à lista de palavras decodificadas. Caso contrário, o token desconhecido (<UNK>) é adicionado à lista.

Após a iteração por todos os tokens, a lista de palavras decodificadas é unida em uma única string, com as palavras separadas por espaços, e essa string é retornada.

In [75]:
def decode_tokens(tokens, int_to_word):
    decoded_words = []
    for token in tokens.squeeze():
        if token.item() in int_to_word:
            decoded_words.append(int_to_word[token.item()])
        else:
            decoded_words.append("<UNK>")
    return ' '.join(decoded_words)

O método abaixo é função de conveniência para a geração de texto. Assim, os parâmetros são o start_text, que é o prompt do usuário, o número máximo de tokens a serem gerados e os dicionário de codificação e decodificação.

Incialmente, é o modelo é colocado em modo de avaliação para desativar camadas que são úteis apenas durante o treinamento, como as camadas de dropout. Além disso, o gradiente também é desativado. Assim, é chamado o método de "encode" que gera um tensor, que corresponde a uma sequência de IDs que correspondem as palavras do prompt, ou sejam , as palavras que darão ínicio à geração de texto. Em seguida, é chamado o método "generate" do modelo, sendo o output desse método os output_ids, os quais foram chamadas de "new_token" no método "generate". Posteriormente, chamda-se o método "decode" que decodifica os id de saída para os tokens correspondentes.


In [76]:
def generate_text(start_text, word_to_int, int_to_word, max_new_tokens=50):
    model.eval()
    with torch.no_grad():
        input_ids = encode(start_text, word_to_int) #inputs_ids no formato de tensores
        generated_tokens = model.generate(input_ids, max_new_tokens)
        return decode_tokens(generated_tokens, int_to_word)

O código abaixo realiza um conjunto de testes para o modelo criado. Assim, é definido uma sequência de sentenças para servirem de start_text para o modelo. Em seguida, percorre-se cada uma das senteças, chamando o método "generate_text" para gerar o texto de saída.

In [77]:
test_sentences = [
    "Sofia era uma menina quieta",
    "Na escola, Sofia não era a criança mais extrovertida,",
]

for sentence in test_sentences:
    # generated_text = generate_text(model, sentence, word_to_int, int_to_word, max_new_tokens=50)
    generated_text = generate_text(sentence, word_to_int, int_to_word, max_new_tokens=50)
    print(f"Input: {sentence}")
    print(f"Generated: {generated_text}")
    print("="*80)

aaaaa: tensor([[   15,    37,    91,    48,    33,    19,   158,   159,   159, 47135,
             7,   165,   104,    99,    83,   102,   104,    12,     7,    69,
           122,    20,   124,    24,   108,   104,    20,    53, 14453, 40589,
           168, 45275,   101,   103,    92,     8,    37,   156,   104,    31,
             7, 32598,   155,    69,   135,   142,   107,    83,     7,   130]])
Input: Sofia era uma menina quieta
Generated: : com clara essa anos livros escritos tornaram tornaram <UNK> , coragem de quais brincavam cada de que , se lia . vezes tinha até de . pais <UNK> <UNK> caderno <UNK> para fase rapidamente mas com distantes de aos , <UNK> reinos se eram própria clássicos brincavam , lutava
aaaaa: tensor([[170,  57,   7, 121, 169,   7, 121, 136,  44,  28,  86,  54,  12,  74,
         136,   7, 128,  46,  47, 128,  52,  54,   7, 117, 118,  20,  52,  69,
         141, 108,  56,  92,  95,  56,  61, 104, 105, 106,  38,  47,  48,  74,
          86,  54,  54,   7,  69,

# Referências

Architeture
https://dugas.ch/artificial_curiosity/GPT_architecture.html

https://keras.io/examples/generative/text_generation_with_miniature_gpt/

https://huggingface.co/learn/nlp-course/chapter7/6

https://debuggercafe.com/text-generation-with-transformers/0

https://debuggercafe.com/text-generation-with-transformers/


https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf